In [1]:
#!/usr/bin/python
import csv
import time
import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
headers = {'Content-Type': 'application/json'}
api_url_base = 'https://public.bitbank.cc'
pair = 'btc_jpy'
period = '1min'

today = "{0:%Y%m%d}".format(datetime.today())

In [3]:
def api_ohlcv():
    api_url = '{0}/{1}/candlestick/{2}/{3}'.format(api_url_base, pair, period, today)
    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        ohlcv = json.loads(response.content.decode('utf-8'))['data']['candlestick'][0]['ohlcv']
        return ohlcv
    else:
        return None

In [4]:
ohlcv = api_ohlcv()
open, high, low, close, volume, timestamp = [],[],[],[],[],[]
for i in ohlcv:
    open.append(i[0])
    high.append(i[1])
    low.append(i[2])
    close.append(i[3])
    volume.append(i[4])
    time_str = str(i[5])
    timestamp.append(datetime.fromtimestamp(int(time_str[:10])).strftime('%Y/%m/%d %H:%M:%M'))

date_time_index = pd.to_datetime(timestamp) # convert to DateTimeIndex type
df = pd.DataFrame({'open': open, 'high': high, 'low': low, 'close': close}, index=date_time_index) # volume is not contained
# adjustment for JST if required
# df.index += pd.offsets.Hour(9) # adjustment for JST

In [5]:
print(df.head(5))
df.to_csv('ohlc.csv') # export as csv

                        open     high      low    close
2019-08-17 00:00:00  1101000  1101001  1101000  1101001
2019-08-17 00:01:01  1101002  1103059  1101002  1103031
2019-08-17 00:02:02  1101375  1101802  1101001  1101001
2019-08-17 00:03:03  1101002  1101002  1100000  1100000
2019-08-17 00:04:04  1100000  1100000  1098925  1098925
